# Install StyleGAN-ADA 

In [ ]:
!git clone https://github.com/dvschultz/stylegan2-ada

In [ ]:
!pip install opencv-python==3.4.13.47

In [ ]:
!ls stylegan2-ada/

In [ ]:
%cd stylegan2-ada
!mkdir downloads
!mkdir datasets

In [ ]:
from IPython.display import Image, display

# Download dataset and Convert dataset to .tfrecords

1. Download Metfaces: https://github.com/NVlabs/metfaces-dataset
   put the dataset into folder. e.g. training_session/datasets/Metfaces
2. Other datasets could find in: https://github.com/NVlabs/stylegan2-ada-pytorch

In [ ]:
# Commands to download the required datsets and convert it to ensorflow formats i.e tf.records 
# Dowload dataset
!wget "https://www.dropbox.com/s/0ybsudabqscstf7/biked_dataset.tar.gz" -q -O biked_dataset.tar.gz
# extract dataset
!tar -zxvf biked_dataset.tar.gz
# Delete the tar.gz file
!rm biked_dataset.tar.gz
!mv biked ../datasets/

In [ ]:
#Example
#update this to the path to your image folder
dataset_path = "../datasets/biked" #"../datasets/MetFaces" #"/home/ubuntu/Desktop/training_session/datasets/MetFaces"
#give your dataset a name
dataset_name = 'biked' #'MetFaces'

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

# Training process

## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
!python train.py --help

### transfer learning -- finetuning from a existing model

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "MetFaces" #"mydataset"
#how often should the model generate samples and a .pkl file
snapshot_count = 2
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = True
#metrics? 
metric_list = None
#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "ffhq1024"

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from}

# Generate images from pre-trained model

In [ ]:
from IPython.display import Image, display, HTML
import base64
import io

def show_local_mp4_video(file_name, width=640, height=480):
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))
def make_img_grid(images,width=360):
    html = []
    for image in images:
        with open(image, "rb") as img_file:
            my_string = base64.b64encode(img_file.read())
            img_uri = "data:image/png;base64," + my_string.decode('utf8')
        html.append('<img src="{}" style="width:{}px;display:inline;margin:1px"/>'.format(img_uri,str(width)))
    return ''.join(html)

In [ ]:
!python generate.py --help

## Options
`--network`: Make sure the `--network` argument points to your .pkl file.

`--seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--trunc`: This sets the truncation amount.This can have a subtle or dramatic affect on your images depending on the value you use. Most people choose between 0.5 and 1.0, but technically it's infinite. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.


`--outdir`:  Where to save the results.

In [ ]:
!python generate.py generate-images\
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl \
  --outdir=results --seeds=6600-6620

In [ ]:
listOfImageNames = ['./results/seed6600.png',
                    './results/seed6601.png',
                    './results/seed6602.png',
                    './results/seed6603.png',
                    './results/seed6604.png',
                    './results/seed6605.png',
                    './results/seed6606.png',
                    './results/seed6608.png',
                    './results/seed6609.png',
                    './results/seed6610.png',
                    './results/seed6611.png',
                    './results/seed6612.png',
                    './results/seed6613.png',
                    './results/seed6614.png',
                    './results/seed6615.png',
                    './results/seed6616.png',
                    './results/seed6617.png',
                   ]

# for imageName in listOfImageNames:
#     display(Image(filename=imageName, width=400))
    

display(HTML(make_img_grid(listOfImageNames, 200)))

In [ ]:
!python generate.py generate-images \
    --outdir=out_child_girl --trunc=0.7 \
    --seeds=1010-1018 --create-grid \
    --network="./pretrain/Girls.pkl"

In [ ]:
display(Image(filename='./out_child_girl/grid.png', width=800))

In [ ]:
!python generate.py generate-images \
    --outdir=out_child_boy --trunc=0.7 \
    --seeds=1000-1008 --create-grid \
    --network="./pretrain/Boys.pkl"

In [ ]:
display(Image(filename='./out_child_boy/grid.png', width=800))

In [ ]:
!python generate.py generate-images \
    --outdir=out_fish --trunc=0.7 \
    --seeds=1000-1008 --create-grid \
    --network=https://github.com/jeffheaton/pretrained-gan-fish/releases/download/1.0.0/fish-gan-2020-12-09.pkl

#display(Image(filename='/home/ubuntu/Desktop/training_session/stylegan2-ada/out_fish/grid.png', width=800))

In [ ]:
display(Image(filename='/home/ubuntu/Desktop/training_session/stylegan2-ada/out_fish/grid.png', width=800))

In [ ]:
!python generate.py generate-images \
    --outdir=out_metfaces --trunc=0.7 \
    --seeds=1000-1008 --create-grid \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl
    #https://github.com/jeffheaton/pretrained-merry-gan-mas/releases/download/v1/christmas-gan-2020-12-03.pkl

In [ ]:

display(Image(filename='/home/ubuntu/Desktop/training_session/stylegan2-ada/out_metfaces/grid.png', width=800))

# Latent space exploration

Transforming the latent vector between two images.
You can create a video that shows the progression through two GAN seeds. This technique creates a very cool "morph" effect.
![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan_progression.jpg "GAN")

In [ ]:
import sys
import pickle
import os
import numpy as np
import PIL.Image
from IPython.display import Image
import matplotlib.pyplot as plt

sys.path.insert(0, "stylegan2-ada")

import dnnlib
import dnnlib.tflib as tflib

def seed2vec(Gs, seed):
  rnd = np.random.RandomState(seed)
  return rnd.randn(1, *Gs.input_shape[1:])

def init_random_state(Gs, seed):
  rnd = np.random.RandomState(seed) 
  noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
  tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]

def display_image(image):
  plt.axis('off')
  plt.imshow(image)
  plt.show()

def generate_image(Gs, z, truncation_psi):
    # Render images for dlatents initialized from random seeds.
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    if truncation_psi is not None:
        Gs_kwargs['truncation_psi'] = truncation_psi

    label = np.zeros([1] + Gs.input_shapes[1][1:])
    images = Gs.run(z, label, **Gs_kwargs) # [minibatch, height, width, channel]
    return images[0]

In [ ]:
def expand_seed(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(seed)
    result.append( rnd.randn(1, vector_size) ) 
  return result

#URL = "https://github.com/jeffheaton/pretrained-gan-fish/releases/download/1.0.0/fish-gan-2020-12-09.pkl"
#URL = "https://github.com/jeffheaton/pretrained-merry-gan-mas/releases/download/v1/christmas-gan-2020-12-03.pkl"
URL = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl"
tflib.init_tf()
print('Loading networks from "%s"...' % URL)
with dnnlib.util.open_url(URL) as fp:
    _G, _D, Gs = pickle.load(fp)

vector_size = Gs.input_shape[1:][0]
# range(8192,8300)
seeds = expand_seed( [8192+1,8192+9], vector_size)
#generate_images(Gs, seeds,truncation_psi=0.5)
print(seeds[0].shape)

In [ ]:
# Choose your seeds to morph through and the number of steps to take to get to each.

SEEDS = [3004,3031,3033,3111,3191,3253]
STEPS = 100

from tqdm.notebook import tqdm

# Remove any prior results
!rm ./results/morph_results/* 
os.makedirs("./results/morph_results", exist_ok=True)

# Generate the images for the video.
idx = 0
for i in range(len(SEEDS)-1):
  v1 = seed2vec(Gs, SEEDS[i])
  v2 = seed2vec(Gs, SEEDS[i+1])

  diff = v2 - v1
  step = diff / STEPS
  current = v1.copy()

  for j in tqdm(range(STEPS), desc=f"Seed {SEEDS[i]}"):
    current = current + step
    init_random_state(Gs, 10)
    img = generate_image(Gs, current, 1.0)
    PIL.Image.fromarray(img, 'RGB').save(f'./results/morph_results/frame-{idx}.png')
    idx+=1
 
# Link the images into a video.
#!ffmpeg -r 30 -i ./results/morph_results/frame-%d.png -vcodec mpeg4 -y movie.mp4


In [ ]:
!ffmpeg -r 30 -i ./results/morph_results/frame-%d.png -vcodec libx264 -pix_fmt yuv420p -y movie.mp4
show_local_mp4_video("movie.mp4", width=512, height=512)

## Other interesting examples.

### Truncation Traversal

Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. Most people choose between 0.5 and 1.0, but technically it's infinite. 

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

###Options 
`--network`: Again, point this to your .pkl file.

`--seed`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

In [ ]:
%pip install opensimplex
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl -O ./pretrain/network.pkl

In [ ]:
!python generate.py truncation-traversal --network="./pretrain/network.pkl" --seed=0 --start=-2.0 --stop=2.0 --increment=0.1 --outdir="./results/tt" --fps=30

In [ ]:
show_local_mp4_video("./results/tt/truncation-traversal-seed0-start-2.0-stop2.0.mp4", width=512, height=512)

## Interpolations
Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at two different examples of interpolation: a linear interpolation and a random noise loop.

Both methods require the following options:

`--network`

`--walk-type`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value

### Linear Interpolation in Z Space     

In [ ]:
!python generate.py generate-latent-walk --network="./pretrain/network.pkl" --walk-type="line-z" --seeds=0,2,5,0 --outdir="./results/z-walk" #--frames 1440

In [ ]:
show_local_mp4_video("./results/z-walk/walk-z-line0-2-5-0-24fps.mp4", width=512, height=512)

### Linear Interpolation in W Space

In [ ]:
!python generate.py generate-latent-walk --network="./pretrain/network.pkl" --walk-type="line-w" --seeds=0,2,5,0 --outdir="./results/w-walk"

In [ ]:
show_local_mp4_video("results/w-walk/walk-w-line0-2-5-0-24fps.mp4", width=512, height=512)

### Noise Loop Interpolation

In [ ]:
!python generate.py generate-latent-walk --network="./pretrain/network.pkl" --walk-type="noiseloop" --start_seed=0 --outdir="results/noise1" --diameter=2.0

In [ ]:
show_local_mp4_video("results/noise1/walk-z-noiseloop-seed0-24fps.mp4", width=512, height=512)

# Reference

1. https://github.com/ArthurFDLR/GANightSky
2. https://towardsdatascience.com/how-to-train-stylegan2-ada-with-custom-dataset-dc268ff70544
3. https://github.com/dvschultz/stylegan2-training
4. https://github.com/jeffheaton/t81_558_deep_learning